In [ ]:
# import header files
%matplotlib inline
import torch
import torch.nn as nn
import torchvision
from functools import partial
from dataclasses import dataclass
from collections import OrderedDict
import glob
import os
import random
import tensorflow as tf
from tensorflow import keras
import numpy as np
import seaborn as sn
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
import time
import copy
import tqdm
import torch
import random
from PIL import Image
import torch.optim as optim
from torchvision import models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset,DataLoader

In [ ]:
# load my google drive
def auth_gdrive():
  from google.colab import drive
  if os.path.exists('content/gdrive/My Drive'): return
  drive.mount('/content/gdrive')
def load_gdrive_dataset():
  loader_assets = 'myepill.zip'
  auth_gdrive()

In [ ]:
# mount my google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
load_gdrive_dataset()

Mounted at /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# unzip dataset
!unzip "/content/gdrive/MyDrive/myepill.zip"

Streaming output truncated to the last 5000 lines.
  inflating: myepill/train/55154-3495_0_0.jpg  
  inflating: myepill/train/55154-3495_0_1.jpg  
  inflating: myepill/train/55154-3516_0_0.jpg  
  inflating: myepill/train/55154-3516_0_1.jpg  
  inflating: myepill/train/55154-3526_0_0.jpg  
  inflating: myepill/train/55154-3526_0_1.jpg  
  inflating: myepill/train/55154-3580_0_0.jpg  
  inflating: myepill/train/55154-3580_0_1.jpg  
  inflating: myepill/train/55154-3581_0_0.jpg  
  inflating: myepill/train/55154-3581_0_1.jpg  
  inflating: myepill/train/55154-3582_0_0.jpg  
  inflating: myepill/train/55154-3582_0_1.jpg  
  inflating: myepill/train/55154-3621_0_0.jpg  
  inflating: myepill/train/55154-3621_0_1.jpg  
  inflating: myepill/train/55154-3622_0_0.jpg  
  inflating: myepill/train/55154-3622_0_1.jpg  
  inflating: myepill/train/55154-3625_0_0.jpg  
  inflating: myepill/train/55154-3625_0_1.jpg  
  inflating: myepill/train/55154-3650_0_0.jpg  
  inflating: myepill/train/55154-3650

In [ ]:
# define transforms
train_transforms = torchvision.transforms.Compose([torchvision.transforms.RandomRotation(30),
                                       torchvision.transforms.Resize((32, 32)),
                                       torchvision.transforms.RandomHorizontalFlip(),
                                       torchvision.transforms.ToTensor(),
                                       torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
# get data
train_data = torchvision.datasets.ImageFolder("/content/myepill/train/", transform=train_transforms)
test_data = torchvision.datasets.ImageFolder("/content/myepill/test/", transform=train_transforms)

In [ ]:
# data loader
trainloader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True, num_workers=1, pin_memory=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True, num_workers=1, pin_memory=True)

In [ ]:
# define the model
class MobileNet(nn.Module):
    def __init__(self, classes = 1000):
        super(MobileNet, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
            )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )

        self.model = nn.Sequential(
            conv_bn(  3,  32, 1), 
            conv_dw( 32,  64, 1),
            conv_dw( 64, 128, 1),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            nn.AvgPool2d(4),
        )
        self.fc = nn.Linear(1024, classes)

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 32*32)
        x = self.fc(x)
        return x

In [ ]:
# print the model
import math
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MobileNet()
model.to(device)

MobileNet(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 128, kernel_size=(1, 1),

In [ ]:
# print summary of the model
from torchvision import models
from torchsummary import summary
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
            Conv2d-4           [-1, 32, 32, 32]             288
       BatchNorm2d-5           [-1, 32, 32, 32]              64
              ReLU-6           [-1, 32, 32, 32]               0
            Conv2d-7           [-1, 64, 32, 32]           2,048
       BatchNorm2d-8           [-1, 64, 32, 32]             128
              ReLU-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 64, 32, 32]             576
      BatchNorm2d-11           [-1, 64, 32, 32]             128
             ReLU-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 32, 32]           8,192
      BatchNorm2d-14          [-1, 128,

In [ ]:
# loss function to be used
criterion = torch.nn.CrossEntropyLoss()
# optimizer to be used
optimizer = torch.optim.SGD(model.parameters(), lr=5e-3, momentum=0.9, weight_decay=5e-4)

In [ ]:
# training step...
from torch.utils.tensorboard import SummaryWriter
train_losses = 0.0
train_accuracy = 0
epochs = 50
for epoch in range(epochs):  # loop over the dataset multiple times
    print('Epoch-{0}:'.format(epoch + 1, optimizer.param_groups[0]['lr']))
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data # get the inputs; data is a list of [inputs, labels]
        inputs, labels = inputs.cuda(), labels.cuda() # for using data in GPU
        optimizer.zero_grad() # zero the parameter gradients
        outputs = model(inputs) # forward
        loss = criterion(outputs, labels) # calculate loss
        loss.backward() # backward loss
        optimizer.step() # optimize gradients
        train_losses += loss.item() # save loss
        _, preds = torch.max(outputs, 1) # save prediction
        train_accuracy += torch.sum(preds == labels.data) # save train_accuracy       
        if i % 1000 == 999:    # every 1000 mini-batches...           
            steps = epoch * len(trainloader) + i # calculate steps 
            batch = i*batch_size # calculate batch 
            # Save train_accuracy and loss to Tensorboard
            writer.add_scalar('Training loss by steps', train_losses / batch, steps)
            writer.add_scalar('Training accuracy by steps', train_accuracy / batch, steps)          
    train_losses = 0.0
    train_accuracy = 0    
print('Train is finished...')

Epoch-1:
Epoch-2:
Epoch-3:
Epoch-4:
Epoch-5:
Epoch-6:
Epoch-7:
Epoch-8:
Epoch-9:
Epoch-10:
Epoch-11:
Epoch-12:
Epoch-13:
Epoch-14:
Epoch-15:
Epoch-16:
Epoch-17:
Epoch-18:
Epoch-19:
Epoch-20:
Epoch-21:
Epoch-22:
Epoch-23:
Epoch-24:
Epoch-25:
Epoch-26:
Epoch-27:
Epoch-28:
Epoch-29:
Epoch-30:
Epoch-31:
Epoch-32:
Epoch-33:
Epoch-34:
Epoch-35:
Epoch-36:
Epoch-37:
Epoch-38:
Epoch-39:
Epoch-40:
Epoch-41:
Epoch-42:
Epoch-43:
Epoch-44:
Epoch-45:
Epoch-46:
Epoch-47:
Epoch-48:
Epoch-49:
Epoch-50:
Train is finished...


In [ ]:
# test step
from torch.utils.tensorboard import SummaryWriter
test_losses = 0.0
test_accuracy = 0
epochs = 50
for epoch in range(epochs):  # loop over the dataset multiple times
    print('Epoch-{0}:'.format(epoch + 1, optimizer.param_groups[0]['lr']))
    for i, data in enumerate(testloader, 0):
        inputs, labels = data # get the inputs; data is a list of [inputs, labels]
        inputs, labels = inputs.cuda(), labels.cuda() # for using data in GPU
        optimizer.zero_grad() # zero the parameter gradients      
        outputs = model(inputs) # forward
        loss = criterion(outputs, labels) # calculate loss
        loss.backward() # backward loss
        optimizer.step() # optimize gradients
        test_losses += loss.item() # save loss
        _, preds = torch.max(outputs, 1) # save prediction
        test_accuracy += torch.sum(preds == labels.data) # save test_accuracy        
        if i % 1000 == 999:    # every 1000 mini-batches...           
            steps = epoch * len(testloader) + i # calculate steps 
            batch = i*batch_size # calculate batch 
            print("Test loss {:.5} Test Accuracy {:.5} Steps: {}".format(test_losses / batch, test_accuracy/batch, steps))           
            # Save test_accuracy and loss to Tensorboard
            writer.add_scalar('Test loss by steps', test_losses / batch, steps)
            writer.add_scalar('Test accuracy by steps', test_accuracy / batch, steps)                      
    print("Test Accuracy: {}/{} ({:.5} %) Test Loss: {:.5}".format(test_accuracy, len(testloader), 100. * test_accuracy / len(testloader.dataset), test_losses / len(testloader.dataset)))   
    test_losses = 0.0
    test_accuracy = 0    
print('Test is Finished...')

Epoch-1:
Test Accuracy: 864/125 (86.4 %) Test Loss: 0.052885
Epoch-2:
Test Accuracy: 936/125 (93.6 %) Test Loss: 0.02708
Epoch-3:
Test Accuracy: 937/125 (93.7 %) Test Loss: 0.022468
Epoch-4:
Test Accuracy: 950/125 (95.0 %) Test Loss: 0.018869
Epoch-5:
Test Accuracy: 955/125 (95.5 %) Test Loss: 0.017865
Epoch-6:
Test Accuracy: 953/125 (95.3 %) Test Loss: 0.017139
Epoch-7:
Test Accuracy: 959/125 (95.9 %) Test Loss: 0.012482
Epoch-8:
Test Accuracy: 971/125 (97.1 %) Test Loss: 0.010487
Epoch-9:
Test Accuracy: 971/125 (97.1 %) Test Loss: 0.010797
Epoch-10:
Test Accuracy: 969/125 (96.9 %) Test Loss: 0.011494
Epoch-11:
Test Accuracy: 976/125 (97.6 %) Test Loss: 0.0093141
Epoch-12:
Test Accuracy: 965/125 (96.5 %) Test Loss: 0.011362
Epoch-13:
Test Accuracy: 968/125 (96.8 %) Test Loss: 0.010534
Epoch-14:
Test Accuracy: 971/125 (97.1 %) Test Loss: 0.011258
Epoch-15:
Test Accuracy: 977/125 (97.7 %) Test Loss: 0.0084506
Epoch-16:
Test Accuracy: 983/125 (98.3 %) Test Loss: 0.007789
Epoch-17:
Test A